In [165]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import os

In [166]:
species = {}

species['00650'] = 'Phosphate, water, unfiltered'
species['00653'] = 'Phosphate, water, filtered'
species['00660'] = 'Orthophosphate, water, filtered'
species['00662'] = 'Phosphorus, water, unfiltered, recoverable'
species['00665'] = 'Phosphorus, water, unfiltered'
species['00666'] = 'Phosphorus, water, filtered'
species['00669'] = 'Hydrolyzable phosphorus, water, unfiltered'
species['00670'] = 'Organic phosphorus, water, unfiltered'
species['00671'] = 'Orthophosphate, water, filtered'
species['00672'] = 'Hydrolyzable phosphorus, water, filtered'
species['00673'] = 'Organic phosphorus, water, filtered'


species['00600'] = 'Total nitrogen [nitrate + nitrite + ammonia + organic-N], water, unfiltered'
species['00602'] = 'Total nitrogen [nitrate + nitrite + ammonia + organic-N], water, filtered'
species['00605'] = 'Organic nitrogen, water, unfiltered'
species['00607'] = 'Organic nitrogen, water, filtered'
species['00608'] = 'Ammonia (NH3 + NH4+), water, filtered'
species['00610'] = 'Ammonia (NH3 + NH4+), water, unfiltered'
species['00613'] = 'Nitrite, water, filtered'
species['00615'] = 'Nitrite, water, unfiltered'
species['00618'] = 'Nitrate, water, filtered'
species['00619'] = 'Ammonia (un-ionized), water, unfiltered, calculated'
species['00620'] = 'Nitrate, water, unfiltered'
species['00623'] = 'Ammonia plus organic nitrogen, water, filtered'
species['00625'] = 'Ammonia plus organic nitrogen, water, unfiltered'
species['00630'] = 'Nitrate plus nitrite, water, unfiltered'
species['00631'] = 'Nitrate plus nitrite, water, filtered'
species['00635'] = 'Ammonia plus organic nitrogen, water, unfiltered'
species['00636'] = 'Ammonia plus organic nitrogen, water, filtered'
species['00639'] = 'Albuminoid nitrogen, water, unfiltered'

species['00954'] = 'Silica, water, unfiltered, recoverable'
species['00955'] = 'Silica, water, filtered'
species['00956'] = 'Silica, water, unfiltered'

species['00940'] = 'Chloride, water, filtered'

In [167]:
basin = 'Western'
river = 'Maumee R'
file_name = 'Maumee-USGS-04193500.csv.zip'


csv_file_df = pd.read_csv(ZipFile('../../measurements/USGS stream data/'+ ("/").join([basin, river, file_name])).open(name=file_name[:-4]))
csv_file_df['ActivityStartDate'] = pd.to_datetime(csv_file_df['ActivityStartDate'], errors='coerce')
database = csv_file_df.ProviderName.unique()[0]

df = pd.DataFrame(columns=['Date', 'MonitoringLocationIdentifier'])

for code in species.keys():
    sp = csv_file_df[(csv_file_df['USGSPCode']==int(code)) & (csv_file_df['ActivityStartDate'].dt.year>1995)][['ActivityStartDate','MonitoringLocationIdentifier','ResultMeasureValue','ResultMeasure/MeasureUnitCode']]
    if sp.size > 1:
        units = sp['ResultMeasure/MeasureUnitCode'].dropna().unique()[0]
        sp.drop(columns=['ResultMeasure/MeasureUnitCode'], inplace=True)
        sp['ResultMeasureValue'] = pd.to_numeric(sp['ResultMeasureValue'])
        sp = sp.groupby(['MonitoringLocationIdentifier','ActivityStartDate']).mean().reset_index()
        sp.rename(columns={'ActivityStartDate':'Date', 'ResultMeasureValue': '{} - {}, {}'.format(code, species[code], units) }, index=str, inplace=True)
        df = pd.merge(df, sp, on=['Date', 'MonitoringLocationIdentifier'], how='outer')
df = df.reindex_axis(sorted(df.columns), axis=1)
df = df.set_index('Date')

if not os.path.exists(("/").join([basin, river, database])):
    os.makedirs(("/").join([basin, river, database]))

df.to_csv(("/").join([basin, river, database, file_name[:-4]]))

/Users/imarkelo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


In [168]:
df.describe()

,"00600 - Total nitrogen [nitrate + nitrite + ammonia + organic-N], water, unfiltered, mg/l","00602 - Total nitrogen [nitrate + nitrite + ammonia + organic-N], water, filtered, mg/l","00605 - Organic nitrogen, water, unfiltered, mg/l","00607 - Organic nitrogen, water, filtered, mg/l","00608 - Ammonia (NH3 + NH4+), water, filtered, mg/l as N","00613 - Nitrite, water, filtered, mg/l as N","00618 - Nitrate, water, filtered, mg/l as N","00623 - Ammonia plus organic nitrogen, water, filtered, mg/l as N","00625 - Ammonia plus organic nitrogen, water, unfiltered, mg/l as N","00631 - Nitrate plus nitrite, water, filtered, mg/l as N","00660 - Orthophosphate, water, filtered, mg/l asPO4","00665 - Phosphorus, water, unfiltered, mg/l as P","00666 - Phosphorus, water, filtered, mg/l as P","00671 - Orthophosphate, water, filtered, mg/l as P","00940 - Chloride, water, filtered, mg/l","00955 - Silica, water, filtered, mg/l"
count,201.000000,176.000000,157.000000,113.000000,164.000000,260.000000,262.000000,194.000000,209.000000,262.000000,230.000000,294.000000,165.000000,230.000000,295.000000,190.000000
mean,5.548507,5.206761,1.288949,0.705664,0.084409,0.041592,4.229414,0.738505,1.389569,4.270254,0.216148,0.226299,0.081515,0.070567,38.716932,4.382447
std,2.993913,2.879170,0.495242,0.273084,0.089085,0.042852,2.594770,0.216987,0.480380,2.619547,0.162015,0.174691,0.054334,0.052772,18.839318,2.757660
min,0.780000,0.670000,0.200000,0.150000,0.007000,0.001000,0.018000,0.120000,0.600000,0.020000,0.011000,0.008000,0.010000,0.004000,5.710000,0.030000
25%,3.100000,2.900000,0.980000,0.550000,0.030000,0.020750,2.177500,0.600000,1.100000,2.217500,0.067000,0.115000,0.030000,0.021000,25.800000,1.900000
50%,5.600000,5.350000,1.200000,0.670000,0.050000,0.030000,4.352500,0.700000,1.300000,4.385000,0.210500,0.170000,0.080000,0.070000,35.200000,5.100000
75%,7.600000,7.025000,1.600000,0.860000,0.100000,0.050000,5.797500,0.817500,1.600000,5.885000,0.328000,0.270000,0.120000,0.107000,48.725000,6.697500
max,16.000000,16.000000,4.400000,2.000000,0.580000,0.368000,13.500000,2.000000,4.600000,13.800000,0.986000,1.500000,0.350000,0.322000,105.000000,11.200000


In [169]:
basin = 'Western'
river = 'Sandusky R'
file_name = 'Sandusky-USGS-410150083125701.csv.zip'


csv_file_df = pd.read_csv(ZipFile('../../measurements/USGS stream data/'+ ("/").join([basin, river, file_name])).open(name=file_name[:-4]))
csv_file_df['ActivityStartDate'] = pd.to_datetime(csv_file_df['ActivityStartDate'], errors='coerce')
database = csv_file_df.ProviderName.unique()[0]

df = pd.DataFrame(columns=['Date', 'MonitoringLocationIdentifier'])

for code in species.keys():
    sp = csv_file_df[(csv_file_df['USGSPCode']==int(code)) & (csv_file_df['ActivityStartDate'].dt.year>1995)][['ActivityStartDate','MonitoringLocationIdentifier','ResultMeasureValue','ResultMeasure/MeasureUnitCode']]
    if sp.size > 1:
        units = sp['ResultMeasure/MeasureUnitCode'].dropna().unique()[0]
        sp.drop(columns=['ResultMeasure/MeasureUnitCode'], inplace=True)
        sp['ResultMeasureValue'] = pd.to_numeric(sp['ResultMeasureValue'])
        sp = sp.groupby(['MonitoringLocationIdentifier','ActivityStartDate']).mean().reset_index()
        sp.rename(columns={'ActivityStartDate':'Date', 'ResultMeasureValue': '{} - {}, {}'.format(code, species[code], units) }, index=str, inplace=True)
        df = pd.merge(df, sp, on=['Date', 'MonitoringLocationIdentifier'], how='outer')
df = df.reindex_axis(sorted(df.columns), axis=1)
df.sort_values(by='Date', inplace=True)
df = df.set_index('Date')

if not os.path.exists(("/").join([basin, river, database])):
    os.makedirs(("/").join([basin, river, database]))

df.to_csv(("/").join([basin, river, database, file_name[:-4]]))

/Users/imarkelo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


In [170]:
df.describe()

,"00605 - Organic nitrogen, water, unfiltered, mg/l","00608 - Ammonia (NH3 + NH4+), water, filtered, mg/l as N","00613 - Nitrite, water, filtered, mg/l as N","00618 - Nitrate, water, filtered, mg/l as N","00631 - Nitrate plus nitrite, water, filtered, mg/l as N","00660 - Orthophosphate, water, filtered, mg/l asPO4","00665 - Phosphorus, water, unfiltered, mg/l as P","00671 - Orthophosphate, water, filtered, mg/l as P","00940 - Chloride, water, filtered, mg/l"
count,10.000000,10.00000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,0.656000,0.03800,0.034833,4.302500,4.345833,0.260750,0.251000,0.085167,24.848333
std,0.303505,0.02974,0.040794,3.701523,3.753681,0.131334,0.176045,0.042791,11.254916
min,0.060000,0.01000,0.007000,1.700000,1.710000,0.100000,0.107000,0.033000,5.480000
25%,0.440000,0.01500,0.014000,2.007500,2.020000,0.140750,0.119750,0.045750,16.550000
50%,0.725000,0.03500,0.023500,3.115000,3.145000,0.269500,0.191500,0.088000,27.050000
75%,0.885000,0.04750,0.036000,4.115000,4.140000,0.361000,0.281750,0.117750,33.100000
max,1.000000,0.11000,0.159000,13.500000,13.700000,0.463000,0.651000,0.151000,42.500000
